In [1]:
import requests
import pandas as pd

In [5]:
# --- Combined SEC Company Facts Table + Plotly Visualizations ---
# Requirements:
#   pip install requests pandas plotly

import plotly.express as px
import plotly.graph_objects as go


def get_company_facts(cik: str):
    cik = str(cik).strip().zfill(10)

    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    headers = {
        "User-Agent": "Asad Capstone Project (shaikasadashraf1@gmail.com)",
        "Accept-Encoding": "gzip, deflate",
        "Host": "data.sec.gov",
    }

    response = requests.get(url, headers=headers, timeout=30)
    response.raise_for_status()
    return response.json()


def _get_fact_df(data: dict, tag: str) -> tuple[pd.DataFrame, str] | tuple[None, None]:
    """Extract a DataFrame for a given US-GAAP tag from SEC companyfacts JSON."""
    facts = data.get("facts", {}).get("us-gaap", {})
    if tag not in facts:
        return None, None

    units = facts[tag].get("units", {})
    if not units:
        return None, None

    unit_key = "USD" if "USD" in units else list(units.keys())[0]
    df = pd.DataFrame(units[unit_key])

    # Normalize dates
    if "end" in df.columns:
        df["end"] = pd.to_datetime(df["end"], errors="coerce")

    # Some items have "fy"/"fp"/"form"/"frame" columns; keep them if present
    return df, unit_key


def display_financial_item(data: dict, tag: str, head_n: int = 10):
    """Display a quick table preview (like your original)."""
    df, unit_key = _get_fact_df(data, tag)

    if df is None or df.empty:
        print(f"{tag} not found for this company.")
        return

    # Sort latest first for display
    if "end" in df.columns:
        df_disp = df.sort_values("end", ascending=False)
    else:
        df_disp = df.copy()

    print(f"\n=== {tag} ({unit_key}) ===")
    display(df_disp.head(head_n))


def build_timeseries_for_tag(
    data: dict,
    tag: str,
    prefer_form: str = "10-K",
    prefer_fp: str = "FY",
    years: int = 12,
):
    """
    Build a clean time-series from companyfacts for a tag.
    Heuristics:
      - Prefer annual FY values from 10-K when available
      - Fallback to whatever is available
      - De-duplicate on date, keep last
      - Keep last `years` observations
    """
    df, unit_key = _get_fact_df(data, tag)
    if df is None or df.empty or "end" not in df.columns or "val" not in df.columns:
        return None, None

    dfx = df.copy()

    # Basic cleanup
    dfx = dfx.dropna(subset=["end", "val"])
    dfx["val"] = pd.to_numeric(dfx["val"], errors="coerce")
    dfx = dfx.dropna(subset=["val"])

    # Prefer annual 10-K FY if possible
    if "form" in dfx.columns:
        dfx_pref = dfx[dfx["form"].astype(str).str.upper() == prefer_form.upper()]
        if not dfx_pref.empty:
            dfx = dfx_pref

    if "fp" in dfx.columns:
        dfx_pref = dfx[dfx["fp"].astype(str).str.upper() == prefer_fp.upper()]
        if not dfx_pref.empty:
            dfx = dfx_pref

    # Sort and de-duplicate
    dfx = dfx.sort_values("end")
    dfx = dfx.drop_duplicates(subset=["end"], keep="last")

    # Keep last N points
    if years is not None and years > 0:
        dfx = dfx.tail(years)

    dfx["tag"] = tag
    return dfx, unit_key


def plot_tag_timeseries(dfx: pd.DataFrame, unit_key: str):
    fig = px.line(
        dfx,
        x="end",
        y="val",
        markers=True,
        title=f"{dfx['tag'].iloc[0]} Over Time ({unit_key})",
        labels={"end": "Period End", "val": unit_key},
    )
    fig.update_layout(hovermode="x unified")
    fig.show()


def plot_multi_tags(data: dict, tags: list[str], years: int = 12):
    """
    Multi-line chart for multiple tags on same plot (best when scales are comparable).
    Also shows each tag individually right after.
    """
    series = []
    unit_key_common = None

    for tag in tags:
        dfx, unit_key = build_timeseries_for_tag(data, tag, years=years)
        if dfx is None or dfx.empty:
            print(f"Skipping {tag}: no usable time series found.")
            continue
        series.append(dfx)
        if unit_key_common is None:
            unit_key_common = unit_key

    if not series:
        print("No series available to plot.")
        return

    all_df = pd.concat(series, ignore_index=True)

    fig = px.line(
        all_df,
        x="end",
        y="val",
        color="tag",
        markers=True,
        title=f"Financials Over Time ({unit_key_common if unit_key_common else 'Units'})",
        labels={"end": "Period End", "val": unit_key_common if unit_key_common else "Value", "tag": "Metric"},
    )
    fig.update_layout(hovermode="x unified")
    fig.show()

    # Individual plots (cleaner for different scales)
    for tag in all_df["tag"].unique():
        sub = all_df[all_df["tag"] == tag].copy()
        plot_tag_timeseries(sub, unit_key_common if unit_key_common else "Value")


def plot_revenue_vs_income(data: dict, years: int = 12):
    rev, rev_unit = build_timeseries_for_tag(data, "Revenues", years=years)
    inc, inc_unit = build_timeseries_for_tag(data, "NetIncomeLoss", years=years)

    if rev is None or inc is None or rev.empty or inc.empty:
        print("Revenue or Net Income not available for comparison.")
        return

    # Align by date (inner join)
    merged = pd.merge(
        rev[["end", "val"]],
        inc[["end", "val"]],
        on="end",
        how="inner",
        suffixes=("_Revenue", "_NetIncome"),
    ).sort_values("end")

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=merged["end"], y=merged["val_Revenue"], mode="lines+markers", name="Revenues"))
    fig.add_trace(go.Scatter(x=merged["end"], y=merged["val_NetIncome"], mode="lines+markers", name="NetIncomeLoss"))

    fig.update_layout(
        title="Revenue vs Net Income (Aligned Period Ends)",
        xaxis_title="Period End",
        yaxis_title="USD",
        hovermode="x unified",
    )
    fig.show()


def plot_margin(data: dict, years: int = 12):
    rev, _ = build_timeseries_for_tag(data, "Revenues", years=years)
    inc, _ = build_timeseries_for_tag(data, "NetIncomeLoss", years=years)

    if rev is None or inc is None or rev.empty or inc.empty:
        print("Revenue or Net Income not available to compute margin.")
        return

    merged = pd.merge(
        rev[["end", "val"]],
        inc[["end", "val"]],
        on="end",
        how="inner",
        suffixes=("_Revenue", "_NetIncome"),
    ).sort_values("end")

    merged["NetMargin"] = merged["val_NetIncome"] / merged["val_Revenue"]

    fig = px.bar(
        merged,
        x="end",
        y="NetMargin",
        title="Net Profit Margin (NetIncomeLoss / Revenues)",
        labels={"end": "Period End", "NetMargin": "Margin"},
    )
    fig.update_layout(hovermode="x unified")
    fig.show()


# --------------------- MAIN RUN --------------------- #
user_cik = input("Enter the company CIK (with or without leading zeros): ")

try:
    data = get_company_facts(user_cik)
    print("\nCompany Name:", data.get("entityName"))
    print("CIK:", data.get("cik"))

    # Your original table displays + Plotly charts
    items = ["Assets", "Liabilities", "Revenues", "NetIncomeLoss"]

    for item in items:
        display_financial_item(data, item)  # table
        dfx, unit_key = build_timeseries_for_tag(data, item, years=12)
        if dfx is not None and not dfx.empty:
            plot_tag_timeseries(dfx, unit_key)
        else:
            print(f"No plottable series for {item}.")

    # Multi-metric plot (all together)
    plot_multi_tags(data, items, years=12)

    # Extra: Revenue vs Income, and Net Margin
    plot_revenue_vs_income(data, years=12)
    plot_margin(data, years=12)

except Exception as e:
    print("Error fetching data:", e)

Enter the company CIK (with or without leading zeros): 320193

Company Name: Apple Inc.
CIK: 320193

=== Assets (USD) ===


,end,val,accn,fy,fp,form,filed,frame
141,2025-12-27,379297000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2025Q4I
140,2025-09-27,359241000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2025Q3I
139,2025-09-27,359241000000,0000320193-25-000079,2025.0,FY,10-K,2025-10-31,NaN
138,2025-06-28,331495000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,CY2025Q2I
137,2025-03-29,331233000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,CY2025Q1I
136,2024-12-28,344085000000,0000320193-25-000008,2025.0,Q1,10-Q,2025-01-31,CY2024Q4I
133,2024-09-28,364980000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,NaN
131,2024-09-28,364980000000,0000320193-24-000123,2024.0,FY,10-K,2024-11-01,NaN
132,2024-09-28,364980000000,0000320193-25-000008,2025.0,Q1,10-Q,2025-01-31,NaN
134,2024-09-28,364980000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,NaN



=== Liabilities (USD) ===


,end,val,accn,fy,fp,form,filed,frame
139,2025-12-27,291107000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2025Q4I
138,2025-09-27,285508000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2025Q3I
137,2025-09-27,285508000000,0000320193-25-000079,2025.0,FY,10-K,2025-10-31,NaN
136,2025-06-28,265665000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,CY2025Q2I
135,2025-03-29,264437000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,CY2025Q1I
134,2024-12-28,277327000000,0000320193-25-000008,2025.0,Q1,10-Q,2025-01-31,CY2024Q4I
131,2024-09-28,308030000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,NaN
129,2024-09-28,308030000000,0000320193-24-000123,2024.0,FY,10-K,2024-11-01,NaN
130,2024-09-28,308030000000,0000320193-25-000008,2025.0,Q1,10-Q,2025-01-31,NaN
132,2024-09-28,308030000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,NaN



=== Revenues (USD) ===


,start,end,val,accn,fy,fp,form,filed,frame
9,2017-10-01,2018-09-29,265595000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018
10,2018-07-01,2018-09-29,62900000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q3
8,2018-04-01,2018-06-30,53265000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q2
7,2017-12-31,2018-03-31,61137000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2018Q1
6,2017-10-01,2017-12-30,88293000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q4
4,2016-09-25,2017-09-30,229234000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017
5,2017-07-02,2017-09-30,52579000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q3
3,2017-04-02,2017-07-01,45408000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q2
2,2017-01-01,2017-04-01,52896000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2017Q1
1,2016-09-25,2016-12-31,78351000000,0000320193-18-000145,2018,FY,10-K,2018-11-05,CY2016Q4



=== NetIncomeLoss (USD) ===


,start,end,val,accn,fy,fp,form,filed,frame
329,2025-09-28,2025-12-27,42097000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2025Q4
328,2024-09-29,2025-09-27,112010000000,0000320193-25-000079,2025.0,FY,10-K,2025-10-31,CY2025
327,2025-03-30,2025-06-28,23434000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,CY2025Q2
326,2024-09-29,2025-06-28,84544000000,0000320193-25-000073,2025.0,Q3,10-Q,2025-08-01,NaN
325,2024-12-29,2025-03-29,24780000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,CY2025Q1
324,2024-09-29,2025-03-29,61110000000,0000320193-25-000057,2025.0,Q2,10-Q,2025-05-02,NaN
323,2024-09-29,2024-12-28,36330000000,0000320193-26-000006,2026.0,Q1,10-Q,2026-01-30,CY2024Q4
322,2024-09-29,2024-12-28,36330000000,0000320193-25-000008,2025.0,Q1,10-Q,2025-01-31,NaN
321,2023-10-01,2024-09-28,93736000000,0000320193-25-000079,2025.0,FY,10-K,2025-10-31,CY2024
320,2023-10-01,2024-09-28,93736000000,0000320193-24-000123,2024.0,FY,10-K,2024-11-01,NaN
